In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.power as sp


def set_annotate (ax, a, p, v): # Процедура для отображения значений
    p=ax[a].patches[p]
    ax[a].annotate(f"{v:.2f}%", (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')

def ttest_result(a, x, y, text):
    tr=ss.ttest_ind(x, y)
    p=tr.pvalue
    f=f"Результат теста на значимость разницы конверсии {text}: statistic={tr.statistic:.3f}, p-value={p:.5f}\n"
    if p<a:
        f+="p-value<alpha -> Конверсии отличаются"
    else:
        f+="p-value>=alpha -> Конверсии не отличаются"
    return f


dataset_link="marketing_AB.csv"
df=pd.read_csv(dataset_link, sep=",")
print (f"Исходный датафрейм:\n{df}\n")
print (f"Типы данных в столбцах:\n{df.dtypes}\n")

# Проверка на пропуски
n_miss=df.isna().sum().sum()
print (f"Найдено {n_miss} пропусков")
if n_miss>0: # Удаление пропусков
    df.dropna(inplace=True)
    print("Пропуски были удалены")

df['converted'] = df['converted'].astype('int') # Преобразуем bool в int для удобства
s_ad=df[df['test group']=='ad']['converted']
s_psa=df[df['test group']=='psa']['converted']

n_users=df.shape[0]
n_ad=s_ad.count()
n_psa=s_psa.count()
n_ad_converted=s_ad.sum()
n_psa_converted=s_psa.sum() 
n_converted=n_ad_converted+n_psa_converted

print (f"\nКоличество уникальных пользователей: {n_users}")
print (f"Количество пользователей, которым была показана реклама ad: {n_ad}")
print (f"Количество пользователей, которым была показана реклама psa: {n_psa}")

print (f"Количество converted-пользователей, которым была показана реклама ad: {n_ad_converted}")
print (f"Количество converted-пользователей, которым была показана реклама psa: {n_psa_converted}")

conv=df['converted'].mean()
conv_ad=n_ad_converted/n_ad
conv_psa=n_psa_converted/n_psa

print (f"Конверсия (всего): {conv:.5f}")
print (f"Конверсия (ad): {conv_ad:.5f}")
print (f"Конверсия (psa): {conv_psa:.5f}")

fig, ax=plt.subplots(ncols=4, figsize=(15, 5), constrained_layout=True)
fig.suptitle('Распределение')
sns.barplot(x=['ad', 'psa'], y = [n_ad, n_psa], ax=ax[0])
sns.barplot(x=['true', 'false'], y = [n_converted, n_users - n_converted], ax=ax[1])
sns.barplot(x=['true', 'false'], y = [n_ad_converted, n_ad - n_ad_converted], ax=ax[2])
sns.barplot(x=['true', 'false'], y = [n_psa_converted, n_psa - n_psa_converted], ax=ax[3])

set_annotate (ax, 0, 0, 100*n_ad/n_users)
set_annotate (ax, 0, 1, 100*n_psa/n_users)
set_annotate (ax, 1, 0, 100*n_converted/n_users)
set_annotate (ax, 1, 1, 100*(1 - n_converted/n_users))

set_annotate (ax, 2, 0, 100*n_ad_converted/n_ad)
set_annotate (ax, 2, 1, 100*(1 - n_ad_converted/n_ad))
set_annotate (ax, 3, 0, 100*n_psa_converted/n_psa)
set_annotate (ax, 3, 1, 100*(1 - n_psa_converted/n_psa))

ax[0].set_xlabel('Тестовые группы')
ax[0].set_ylabel('Количество пользователей')
ax[1].set_xlabel('Конверсия (всего)')
ax[1].set_ylabel('Количество пользователей')
ax[2].set_xlabel('Конверсия (ad)')
ax[2].set_ylabel('Количество пользователей')
ax[3].set_xlabel('Конверсия (psa)')
ax[3].set_ylabel('Количество пользователей')
plt.show()

print ("Предполагаем, что конверсия в группе ad выше, но необходимо провести статистический тест")
confidence=0.95 # Уровень доверия
print (f"\nУровень доверия: {confidence:.2f}")
alpha=np.round(1-confidence, 2)
print (f"Уровень значимости (вероятность ошибки первого рода): {alpha:.2f}")
power=0.8
print (f"Мощность теста: {power:.2f}")
effect=0.10
print (f"Ожидаемый эффект: {effect:.2f}")
size=sp.TTestIndPower().solve_power(effect, power=power, alpha=alpha)
print (f"Минимально достаточный размер выборки: {size:.0f}")
print ("Используемые выборки удовлетворяют данному требованию для получения необходимой статистической мощности")

print (ttest_result(alpha, s_ad, s_psa, "между группами ad и psa"))
print ("Таким образом, тип рекламы ad оказался более эффективным")

print ("\nДополнительно оценим эффективность рекламы в зависимости от дней недели, в которые человек увидел наибольшее ее количество")
print (f"Убедимся, что в столбце 'most ads day' содержатся только дни недели: {df['most ads day'].unique()}")
print (f"Распределение рекламы по дням недели (%):\n{100*df['most ads day'].value_counts(normalize=True)}")

# Вставка столбца, разделяющего выборки
df.insert(0, column='weekend', value=0)
df.loc[(df['most ads day']=='Saturday') | (df['most ads day']=='Sunday'), 'weekend']=1

s_ad_weekend=df[(df['test group']=='ad') & (df['weekend']==1)]['converted']
s_psa_weekend=df[(df['test group']=='psa') & (df['weekend']==1)]['converted']
s_ad_workday=df[(df['test group']=='ad') & (df['weekend']==0)]['converted']
s_psa_workday=df[(df['test group']=='psa') & (df['weekend']==0)]['converted']

print()
print (f"Конверсия (ad, weekend): {s_ad_weekend.mean():.5f}")
print (f"Конверсия (psa, weekend): {s_psa_weekend.mean():.5f}")
print (f"Конверсия (ad, workday): {s_ad_workday.mean():.5f}")
print (f"Конверсия (psa, workday): {s_psa_workday.mean():.5f}")

print ("Обращаем внимание на то, что в рабочие дни конверсия в обеих группах немного выше среднего")

print()
print (f"Размер выборки (ad, weekend): {s_ad_weekend.count():.0f}")
print (f"Размер выборки (psa, weekend): {s_psa_weekend.count():.0f}")
print (f"Размер выборки (ad, workday): {s_ad_workday.count():.0f}")
print (f"Размер выборки (psa, workday): {s_psa_workday.count():.0f}")
print ("Используемые выборки удовлетворяют требованию, описанному ранее")

print()
print (ttest_result(alpha, s_ad_weekend, s_psa_weekend, "между группами (ad, weekend) и (psa, weekend)"))
print (ttest_result(alpha, s_ad_workday, s_psa_workday, "между группами (ad, workday) и (psa, workday)"))

# Визуализация
df_heatmap=df.groupby(['test group', 'weekend'], sort=False)['converted'].agg('mean').mul(100).unstack()
sns.heatmap(data=df_heatmap, annot=True, fmt='.3f', xticklabels=['будни', 'выходные'])
plt.title ('Тепловая карта')
plt.xlabel('Дни недели')
plt.ylabel('Группа')
plt.show()

print ("Заключение: во всех рассмотренных случаях тип рекламы ad оказался более успешным. Наибольшая эффективность отмечена в рабочие дни")